In [3]:
import pandas as pd
import numpy as np
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
import time
import re
from bs4 import BeautifulSoup 
import random 
import multiprocessing
from tqdm import tqdm


df = pd.read_csv('./제주특별자치도음식점목록(통합).csv', sep=',', encoding='CP949')
df['naver_map_url'] = '' # 미리 url을 담을 column을 만들어줌

chromedriver = './chromedriver' 
driver = webdriver.Chrome(chromedriver) 


# driver = webdriver.Chrome(executable_path=r'C:\\Users\\ui\\Desktop\chromedriver.exe') # 웹드라이버가 설치된 경로를 지정해주시면 됩니다.

In [4]:
df

,사업장명,업종구분대분류,업종구분소분류,인허가일자,인허가취소일자,영업상태명,상세영업상태명,폐업일자,휴업시작일자,휴업종료일자,재개업일자,소재지면적,소재지전체주소,도로명전체주소,도로명우편번호,데이터갱신일자,naver_map_url
0,카페송키,일반음식점,기타,20181129,NaN,영업/정상,영업,NaN,NaN,NaN,NaN,80.14,제주특별자치도 제주시 삼도이동 1120-2번지,"제주특별자치도 제주시 무근성안길 16, 1층 (삼도이동)",63166.0,2018-12-01 2:20,
1,그림책방앤카페노란우산,일반음식점,기타,20181227,NaN,영업/정상,영업,NaN,NaN,NaN,NaN,104.25,제주특별자치도 제주시 애월읍 광령리 2981-1번지 자연숲,"제주특별자치도 제주시 애월읍 하광로 515, 104동 102호 (자연숲)",63062.0,2019-01-27 2:40,
2,에릭스에스프레소,일반음식점,기타,20181102,NaN,영업/정상,영업,NaN,NaN,NaN,NaN,27,제주특별자치도 제주시 구좌읍 세화리 1397-6번지,"제주특별자치도 제주시 구좌읍 구좌로 77, 1층",63361.0,2018-11-04 2:36,
3,일품순두부한림점,일반음식점,한식,20190107,NaN,영업/정상,영업,NaN,NaN,NaN,NaN,152,제주특별자치도 제주시 한림읍 대림리 1845-2번지,"제주특별자치도 제주시 한림읍 한림상로 237, 1층",63030.0,2019-01-17 2:40,
4,팔도실비집아라점,일반음식점,식육(숯불구이),20190107,NaN,영업/정상,영업,NaN,NaN,NaN,NaN,82,제주특별자치도 제주시 아라일동 6139,"제주특별자치도 제주시 중앙로 581, 에이동 1층 (아라일동)",63248.0,2020-09-13 2:40,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42597,커피전문점,휴게음식점,푸드트럭,20181214,NaN,폐업,폐업,20201130.0,NaN,NaN,NaN,8.89,제주특별자치도 서귀포시 동홍동 2142-7 도로,NaN,NaN,2020-12-12 2:40,
42598,아름다운서귀포,휴게음식점,푸드트럭,20181112,NaN,폐업,폐업,20201031.0,NaN,NaN,NaN,12.21,제주특별자치도 서귀포시 서홍동 696 앞도로,NaN,NaN,2020-11-20 2:40,
42599,붕어빵과오뎅,휴게음식점,기타 휴게음식점,20181123,NaN,폐업,폐업,20201202.0,NaN,NaN,NaN,6.48,제주특별자치도 서귀포시 성산읍 고성리 1197-7 생고기부위별판매장,"제주특별자치도 서귀포시 성산읍 고성동서로56번길 15, 생고기부위별판매장",63640.0,2020-12-04 2:40,
42600,인정김밥,휴게음식점,일반조리판매,20210121,NaN,폐업,폐업,20210524.0,NaN,NaN,NaN,35,제주특별자치도 서귀포시 안덕면 서광리 1412,"제주특별자치도 서귀포시 안덕면 녹차분재로 56, 다동",63521.0,2021-05-26 2:40,


In [5]:
# 폐업한 가게는 삭제
df = df[df.영업상태명 != '폐업']

In [6]:
df = df.reset_index()

In [7]:
del df['index']

In [8]:
# 데이터프레임 전체를 1/n으로 슬라이싱하여 크롤링

df = df.iloc[6384:9576]
# 6384~9576


In [9]:
df = df.reset_index()
del df['index']

In [10]:
df

,사업장명,업종구분대분류,업종구분소분류,인허가일자,인허가취소일자,영업상태명,상세영업상태명,폐업일자,휴업시작일자,휴업종료일자,재개업일자,소재지면적,소재지전체주소,도로명전체주소,도로명우편번호,데이터갱신일자,naver_map_url
0,수상한소금밭,일반음식점,경양식,20121206,NaN,영업/정상,영업,NaN,NaN,NaN,NaN,50.4,제주특별자치도 제주시 구좌읍 종달리 814-129번지 에이동(1층),제주특별자치도 제주시 구좌읍 종달동길 36-10 (에이동),63364.0,2018-08-31 23:59,
1,이스트힐스카페테리아,일반음식점,한식,20121207,NaN,영업/정상,영업,NaN,NaN,NaN,NaN,147.6,제주특별자치도 제주시 조천읍 와흘리 835 외2필지(1층),제주특별자치도 제주시 조천읍 종인내길 323,63344.0,2020-11-04 2:40,
2,포그린,일반음식점,한식,20121207,NaN,영업/정상,영업,NaN,NaN,NaN,NaN,13.12,NaN,"제주특별자치도 제주시 서광로 249, 3층 (삼도일동)",63182.0,2018-08-31 23:59,
3,큰맘할매순대국밥,일반음식점,한식,20121210,NaN,영업/정상,영업,NaN,NaN,NaN,NaN,105,제주특별자치도 제주시 아라이동 3011-20 (1층),제주특별자치도 제주시 구남동5길 42 (아라이동),63230.0,2020-10-25 2:40,
4,하르방국수,일반음식점,한식,20130530,NaN,영업/정상,영업,NaN,NaN,NaN,NaN,51.96,제주특별자치도 제주시 일도이동 1034-16,제주특별자치도 제주시 삼성로 69 (일도이동),63268.0,2020-09-13 2:40,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3187,돈복편백,일반음식점,한식,20210402,NaN,영업/정상,영업,NaN,NaN,NaN,NaN,75.39,제주특별자치도 제주시 애월읍 애월리 1840-1,"제주특별자치도 제주시 애월읍 애월해안로 29, 1층",63043.0,2021-04-29 2:40,
3188,유로코피자제주2호점,일반음식점,기타,20210402,NaN,영업/정상,영업,NaN,NaN,NaN,NaN,105.5,제주특별자치도 제주시 애월읍 하귀2리 2626,"제주특별자치도 제주시 애월읍 애월해안로 960, 2층",63050.0,2021-04-07 2:40,
3189,오씨엔제주,일반음식점,기타,20210402,NaN,영업/정상,영업,NaN,NaN,NaN,NaN,845.45,제주특별자치도 제주시 한림읍 협재리 2838 라온프라이빗타운,"제주특별자치도 제주시 한림읍 금능남로 127, 라온프라이빗타운 601동 2층 206호",63015.0,2021-04-04 0:23,
3190,카페뷰,일반음식점,기타,20210402,NaN,영업/정상,영업,NaN,NaN,NaN,NaN,85.4,제주특별자치도 제주시 우도면 연평리 752,"제주특별자치도 제주시 우도면 우도해안길 684, 1층",63365.0,2021-04-04 0:23,


In [11]:
# 도로명주소를 , 기준으로 split()하여 사용함 (검색결과의 정확성 높이기 위해)

address_list = df.도로명전체주소.str.split(',').str[0].tolist()
print(address_list)

['제주특별자치도 제주시 구좌읍 종달동길 36-10 (에이동)', '제주특별자치도 제주시 조천읍 종인내길 323', '제주특별자치도 제주시 서광로 249', '제주특별자치도 제주시 구남동5길 42 (아라이동)', '제주특별자치도 제주시 삼성로 69 (일도이동)', '제주특별자치도 제주시 구좌읍 세화1길 20-17', '제주특별자치도 제주시 청귤로3길 50 (이도이동)', '제주특별자치도 제주시 우정로7길 30', '제주특별자치도 제주시 한경면 낙수로 85', '제주특별자치도 제주시 고마로 29', '제주특별자치도 제주시 구좌읍 해녀박물관길 29', '제주특별자치도 제주시 구좌읍 해맞이해안로 1172', '제주특별자치도 제주시 기와4길 11', '제주특별자치도 제주시 테우해안로 172-15', '제주특별자치도 제주시 명림로 575-107', '제주특별자치도 제주시 도남로 62', '제주특별자치도 제주시 신산로 42 (일도이동)', '제주특별자치도 제주시 신성로13길 15 (이도이동)', '제주특별자치도 제주시 애월읍 하귀12길 22', '제주특별자치도 제주시 성신로 15', '제주특별자치도 제주시 연화로4길 2', '제주특별자치도 제주시 한림읍 한림중앙로 46', '제주특별자치도 제주시 비월길 3', '제주특별자치도 제주시 용화로 62 (용담이동)', '제주특별자치도 제주시 서광로32길 23', '제주특별자치도 제주시 도령로 11 (노형동)', '제주특별자치도 제주시 일주동로 350-2', '제주특별자치도 제주시 진동로3길 34', '제주특별자치도 제주시 중앙로 27-26', '제주특별자치도 제주시 애월읍 고하상로 125-1', '제주특별자치도 제주시 국기로4길 16', '제주특별자치도 제주시 중앙로7길 3 (일도일동', '제주특별자치도 제주시 태성로2길 19 (일도이동)', '제주특별자치도 제주시 성신로1길 11 (연동', '제주특별자치도 제주시 용담로 41 (용담삼동)', '제주특별자치도 제주시 진서로 33-3 (화북일동)', '제주특별자치도 제주시 중앙로 

In [12]:
df['주소'] = address_list
df

,사업장명,업종구분대분류,업종구분소분류,인허가일자,인허가취소일자,영업상태명,상세영업상태명,폐업일자,휴업시작일자,휴업종료일자,재개업일자,소재지면적,소재지전체주소,도로명전체주소,도로명우편번호,데이터갱신일자,naver_map_url,주소
0,수상한소금밭,일반음식점,경양식,20121206,NaN,영업/정상,영업,NaN,NaN,NaN,NaN,50.4,제주특별자치도 제주시 구좌읍 종달리 814-129번지 에이동(1층),제주특별자치도 제주시 구좌읍 종달동길 36-10 (에이동),63364.0,2018-08-31 23:59,,제주특별자치도 제주시 구좌읍 종달동길 36-10 (에이동)
1,이스트힐스카페테리아,일반음식점,한식,20121207,NaN,영업/정상,영업,NaN,NaN,NaN,NaN,147.6,제주특별자치도 제주시 조천읍 와흘리 835 외2필지(1층),제주특별자치도 제주시 조천읍 종인내길 323,63344.0,2020-11-04 2:40,,제주특별자치도 제주시 조천읍 종인내길 323
2,포그린,일반음식점,한식,20121207,NaN,영업/정상,영업,NaN,NaN,NaN,NaN,13.12,NaN,"제주특별자치도 제주시 서광로 249, 3층 (삼도일동)",63182.0,2018-08-31 23:59,,제주특별자치도 제주시 서광로 249
3,큰맘할매순대국밥,일반음식점,한식,20121210,NaN,영업/정상,영업,NaN,NaN,NaN,NaN,105,제주특별자치도 제주시 아라이동 3011-20 (1층),제주특별자치도 제주시 구남동5길 42 (아라이동),63230.0,2020-10-25 2:40,,제주특별자치도 제주시 구남동5길 42 (아라이동)
4,하르방국수,일반음식점,한식,20130530,NaN,영업/정상,영업,NaN,NaN,NaN,NaN,51.96,제주특별자치도 제주시 일도이동 1034-16,제주특별자치도 제주시 삼성로 69 (일도이동),63268.0,2020-09-13 2:40,,제주특별자치도 제주시 삼성로 69 (일도이동)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3187,돈복편백,일반음식점,한식,20210402,NaN,영업/정상,영업,NaN,NaN,NaN,NaN,75.39,제주특별자치도 제주시 애월읍 애월리 1840-1,"제주특별자치도 제주시 애월읍 애월해안로 29, 1층",63043.0,2021-04-29 2:40,,제주특별자치도 제주시 애월읍 애월해안로 29
3188,유로코피자제주2호점,일반음식점,기타,20210402,NaN,영업/정상,영업,NaN,NaN,NaN,NaN,105.5,제주특별자치도 제주시 애월읍 하귀2리 2626,"제주특별자치도 제주시 애월읍 애월해안로 960, 2층",63050.0,2021-04-07 2:40,,제주특별자치도 제주시 애월읍 애월해안로 960
3189,오씨엔제주,일반음식점,기타,20210402,NaN,영업/정상,영업,NaN,NaN,NaN,NaN,845.45,제주특별자치도 제주시 한림읍 협재리 2838 라온프라이빗타운,"제주특별자치도 제주시 한림읍 금능남로 127, 라온프라이빗타운 601동 2층 206호",63015.0,2021-04-04 0:23,,제주특별자치도 제주시 한림읍 금능남로 127
3190,카페뷰,일반음식점,기타,20210402,NaN,영업/정상,영업,NaN,NaN,NaN,NaN,85.4,제주특별자치도 제주시 우도면 연평리 752,"제주특별자치도 제주시 우도면 우도해안길 684, 1층",63365.0,2021-04-04 0:23,,제주특별자치도 제주시 우도면 우도해안길 684


In [13]:
# 정정코드
# 사업장명 + 도로명주소 = '사업장명add'
# 지번주소는 정확하지 않은 주소가 많아서 도로명 주소 사용함

df["사업장명add"]=df['사업장명']+""+df['주소'] 

In [14]:
# 정정한 코드

df = df[['사업장명','사업장명add', '업종구분대분류', '업종구분소분류', '주소', 'naver_map_url']]

In [15]:
df

,사업장명,사업장명add,업종구분대분류,업종구분소분류,주소,naver_map_url
0,수상한소금밭,수상한소금밭제주특별자치도 제주시 구좌읍 종달동길 36-10 (에이동),일반음식점,경양식,제주특별자치도 제주시 구좌읍 종달동길 36-10 (에이동),
1,이스트힐스카페테리아,이스트힐스카페테리아제주특별자치도 제주시 조천읍 종인내길 323,일반음식점,한식,제주특별자치도 제주시 조천읍 종인내길 323,
2,포그린,포그린제주특별자치도 제주시 서광로 249,일반음식점,한식,제주특별자치도 제주시 서광로 249,
3,큰맘할매순대국밥,큰맘할매순대국밥제주특별자치도 제주시 구남동5길 42 (아라이동),일반음식점,한식,제주특별자치도 제주시 구남동5길 42 (아라이동),
4,하르방국수,하르방국수제주특별자치도 제주시 삼성로 69 (일도이동),일반음식점,한식,제주특별자치도 제주시 삼성로 69 (일도이동),
...,...,...,...,...,...,...
3187,돈복편백,돈복편백제주특별자치도 제주시 애월읍 애월해안로 29,일반음식점,한식,제주특별자치도 제주시 애월읍 애월해안로 29,
3188,유로코피자제주2호점,유로코피자제주2호점제주특별자치도 제주시 애월읍 애월해안로 960,일반음식점,기타,제주특별자치도 제주시 애월읍 애월해안로 960,
3189,오씨엔제주,오씨엔제주제주특별자치도 제주시 한림읍 금능남로 127,일반음식점,기타,제주특별자치도 제주시 한림읍 금능남로 127,
3190,카페뷰,카페뷰제주특별자치도 제주시 우도면 우도해안길 684,일반음식점,기타,제주특별자치도 제주시 우도면 우도해안길 684,


In [16]:
for i, keyword in enumerate(df['사업장명add'].tolist()):  #정정코드
    print("이번에 찾을 키워드 :", i, f"/ {df.shape[0]} 행",keyword)
    
    try:
        naver_map_search_url = f"https://map.naver.com/v5/search/{keyword}/place" # 검색 url 만들기
        driver.get(naver_map_search_url) # 검색 url 접속, 즉 검색하기
        
        time.sleep(2) # 중요함
        cu = driver.current_url # 검색이 성공된 플레이스에 대한 개별 페이지
        
        res_code = re.findall(r"place/(\d+)", cu)
        final_url = 'https://pcmap.place.naver.com/restaurant/'+res_code[0]+'/review/visitor#'
        
        print(final_url)
        df['naver_map_url'][i]=final_url
        
    except IndexError:
        df['naver_map_url'][i]= ''
        print('none')

이번에 찾을 키워드 : 0 / 3192 행 수상한소금밭제주특별자치도 제주시 구좌읍 종달동길 36-10 (에이동)


C:\Users\ui\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3437: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exec(code_obj, self.user_global_ns, self.user_ns)


none
이번에 찾을 키워드 : 1 / 3192 행 이스트힐스카페테리아제주특별자치도 제주시 조천읍 종인내길 323
https://pcmap.place.naver.com/restaurant/1309166134/review/visitor#
이번에 찾을 키워드 : 2 / 3192 행 포그린제주특별자치도 제주시 서광로 249
none
이번에 찾을 키워드 : 3 / 3192 행 큰맘할매순대국밥제주특별자치도 제주시 구남동5길 42 (아라이동)
https://pcmap.place.naver.com/restaurant/1987422309/review/visitor#
이번에 찾을 키워드 : 4 / 3192 행 하르방국수제주특별자치도 제주시 삼성로 69 (일도이동)
none
이번에 찾을 키워드 : 5 / 3192 행 부가네얼큰이세화점제주특별자치도 제주시 구좌읍 세화1길 20-17
none
이번에 찾을 키워드 : 6 / 3192 행 제주통큰장어제주특별자치도 제주시 청귤로3길 50 (이도이동)
https://pcmap.place.naver.com/restaurant/32782551/review/visitor#
이번에 찾을 키워드 : 7 / 3192 행 포야호제주특별자치도 제주시 우정로7길 30
none
이번에 찾을 키워드 : 8 / 3192 행 휴식공간제주특별자치도 제주시 한경면 낙수로 85
https://pcmap.place.naver.com/restaurant/1565713915/review/visitor#
이번에 찾을 키워드 : 9 / 3192 행 숙이네국수제주특별자치도 제주시 고마로 29
https://pcmap.place.naver.com/restaurant/36835989/review/visitor#
이번에 찾을 키워드 : 10 / 3192 행 세화리움제주특별자치도 제주시 구좌읍 해녀박물관길 29
https://pcmap.place.naver.com/restaurant/37687260/review/visitor#
이번에 찾을 키워드 : 11 / 3192 행 평대성게국수제

none
이번에 찾을 키워드 : 86 / 3192 행 미미이자카야제주특별자치도 제주시 우정로11길 5
https://pcmap.place.naver.com/restaurant/1594615738/review/visitor#
이번에 찾을 키워드 : 87 / 3192 행 다마제주특별자치도 제주시 동광로21길 41
none
이번에 찾을 키워드 : 88 / 3192 행 만나고쪽갈비제주노형점제주특별자치도 제주시 다랑곶4길 14
https://pcmap.place.naver.com/restaurant/1345720035/review/visitor#
이번에 찾을 키워드 : 89 / 3192 행 명가천지연무태장어제주특별자치도 제주시 아연로 438-22
https://pcmap.place.naver.com/restaurant/394082392/review/visitor#
이번에 찾을 키워드 : 90 / 3192 행 애월포구횟집제주특별자치도 제주시 애월읍 애월로11길 22
https://pcmap.place.naver.com/restaurant/1970930937/review/visitor#
이번에 찾을 키워드 : 91 / 3192 행 연어도둑제주특별자치도 제주시 박성내서길 8
https://pcmap.place.naver.com/restaurant/1743481734/review/visitor#
이번에 찾을 키워드 : 92 / 3192 행 어멍네제주특별자치도 제주시 한경면 불그못로 99
https://pcmap.place.naver.com/restaurant/13467780/review/visitor#
이번에 찾을 키워드 : 93 / 3192 행 이서순대국밥제주특별자치도 제주시 한림읍 한림해안로 162
https://pcmap.place.naver.com/restaurant/37470399/review/visitor#
이번에 찾을 키워드 : 94 / 3192 행 소주앤비어제주특별자치도 제주시 흥운길 33 (용담이동)
https://pcmap.place.naver.com/res

none
이번에 찾을 키워드 : 173 / 3192 행 낭만노가리용담점제주특별자치도 제주시 용담로 50 (용담이동)
https://pcmap.place.naver.com/restaurant/1209330736/review/visitor#
이번에 찾을 키워드 : 174 / 3192 행 국수강산제주특별자치도 제주시 신산로 91 (일도이동)
none
이번에 찾을 키워드 : 175 / 3192 행 월랑봉제주특별자치도 제주시 구좌읍 구좌로 25
https://pcmap.place.naver.com/restaurant/21761094/review/visitor#
이번에 찾을 키워드 : 176 / 3192 행 뼈대감제주특별자치도 제주시 한림읍 한림중앙로 61-2
https://pcmap.place.naver.com/restaurant/16862893/review/visitor#
이번에 찾을 키워드 : 177 / 3192 행 nan
none
이번에 찾을 키워드 : 178 / 3192 행 하이티파니제주특별자치도 제주시 연북로 26 (노형동
none
이번에 찾을 키워드 : 179 / 3192 행 화북코끼리만두제주특별자치도 제주시 진동로5길 34 (화북일동
none
이번에 찾을 키워드 : 180 / 3192 행 금성문제주특별자치도 제주시 조천읍 신북로 400
https://pcmap.place.naver.com/restaurant/16887544/review/visitor#
이번에 찾을 키워드 : 181 / 3192 행 괸당네파전제주특별자치도 제주시 동광로6길 13 (이도이동)
https://pcmap.place.naver.com/restaurant/21761452/review/visitor#
이번에 찾을 키워드 : 182 / 3192 행 프라이드프라이드제주특별자치도 제주시 진남로6길 11 (화북일동)
https://pcmap.place.naver.com/restaurant/36442049/review/visitor#
이번에 찾을 키워드 : 183 / 3192 행 전주콩나루콩나물

...

https://pcmap.place.naver.com/restaurant/1983714498/review/visitor#
이번에 찾을 키워드 : 2859 / 3192 행 맛좋은한림통돼지바베큐제주특별자치도 제주시 한림읍 한림로 682
https://pcmap.place.naver.com/restaurant/1795239276/review/visitor#
이번에 찾을 키워드 : 2860 / 3192 행 놀부부대찌개닭갈비제주특별자치도 제주시 서사로 172
none
이번에 찾을 키워드 : 2861 / 3192 행 엄마의제주제주특별자치도 제주시 구좌읍 김녕로 118-10
https://pcmap.place.naver.com/restaurant/1274158340/review/visitor#
이번에 찾을 키워드 : 2862 / 3192 행 장땡정식뷔페제주특별자치도 제주시 우도면 우도해안길 46
https://pcmap.place.naver.com/restaurant/1396937816/review/visitor#
이번에 찾을 키워드 : 2863 / 3192 행 애월돈찜닭제주특별자치도 제주시 애월읍 애월해안로 56
https://pcmap.place.naver.com/restaurant/1176890070/review/visitor#
이번에 찾을 키워드 : 2864 / 3192 행 봉이네한림제주특별자치도 제주시 한림읍 한림남1길 1
https://pcmap.place.naver.com/restaurant/1766755064/review/visitor#
이번에 찾을 키워드 : 2865 / 3192 행 윤가네흑돼지대패삼겹살제주특별자치도 제주시 조천읍 신북로 572
https://pcmap.place.naver.com/restaurant/16887117/review/visitor#
이번에 찾을 키워드 : 2866 / 3192 행 삼선해물탕제주특별자치도 제주시 무근성7길 15
https://pcmap.place.naver.com/restaurant/1426171269/review

https://pcmap.place.naver.com/restaurant/1132346150/review/visitor#
이번에 찾을 키워드 : 2940 / 3192 행 이룸제주특별자치도 제주시 고마로 140
none
이번에 찾을 키워드 : 2941 / 3192 행 어살아있네제주특별자치도 제주시 한경면 신창3길 6
https://pcmap.place.naver.com/restaurant/1114147101/review/visitor#
이번에 찾을 키워드 : 2942 / 3192 행 사부작10제주특별자치도 제주시 한림읍 금능5길 21
https://pcmap.place.naver.com/restaurant/1715598509/review/visitor#
이번에 찾을 키워드 : 2943 / 3192 행 엄마의집밥제주특별자치도 제주시 삼성로5길 20-2
https://pcmap.place.naver.com/restaurant/1313425074/review/visitor#
이번에 찾을 키워드 : 2944 / 3192 행 부성식당제주특별자치도 제주시 한림읍 한수풀로4길 10
none
이번에 찾을 키워드 : 2945 / 3192 행 처갓집양념치킨함덕점제주특별자치도 제주시 조천읍 함덕남16길 108
https://pcmap.place.naver.com/restaurant/1653692370/review/visitor#
이번에 찾을 키워드 : 2946 / 3192 행 제주해물상회제주특별자치도 제주시 한림읍 한림로 16-3
https://pcmap.place.naver.com/restaurant/1436114038/review/visitor#
이번에 찾을 키워드 : 2947 / 3192 행 그래그래삼양점제주특별자치도 제주시 일주동로 349
https://pcmap.place.naver.com/restaurant/1738018292/review/visitor#
이번에 찾을 키워드 : 2948 / 3192 행 제주사랑푸드제주특별자치도 제주시 절물1길 24
https://pcma

https://pcmap.place.naver.com/restaurant/1322177985/review/visitor#
이번에 찾을 키워드 : 3018 / 3192 행 대패명가제주특별자치도 제주시 연수로5길 20
https://pcmap.place.naver.com/restaurant/1598646705/review/visitor#
이번에 찾을 키워드 : 3019 / 3192 행 정우솥뚜껑삼겹살제주특별자치도 제주시 진남로6길 35
https://pcmap.place.naver.com/restaurant/1445516948/review/visitor#
이번에 찾을 키워드 : 3020 / 3192 행 엠박스제주제주특별자치도 제주시 신대로16길 42
https://pcmap.place.naver.com/restaurant/1320831775/review/visitor#
이번에 찾을 키워드 : 3021 / 3192 행 조옹달리제주특별자치도 제주시 구좌읍 해맞이해안로 2290
none
이번에 찾을 키워드 : 3022 / 3192 행 캘리포니아그릴제주특별자치도 제주시 우도면 우도해안길 130-9
https://pcmap.place.naver.com/restaurant/1919394141/review/visitor#
이번에 찾을 키워드 : 3023 / 3192 행 무공돈까스제주이도점제주특별자치도 제주시 남광북1길 1
https://pcmap.place.naver.com/restaurant/1185577467/review/visitor#
이번에 찾을 키워드 : 3024 / 3192 행 제라진감자탕제주특별자치도 제주시 신대로14길 41
https://pcmap.place.naver.com/restaurant/1847005331/review/visitor#
이번에 찾을 키워드 : 3025 / 3192 행 동수네흑돼지제주특별자치도 제주시 애월읍 중엄3길 65
https://pcmap.place.naver.com/restaurant/1750287504/review/visitor#

none
이번에 찾을 키워드 : 3097 / 3192 행 플래닛제주제주특별자치도 제주시 삼무로 35
https://pcmap.place.naver.com/restaurant/1049190960/review/visitor#
이번에 찾을 키워드 : 3098 / 3192 행 300킬로그램제주특별자치도 제주시 연수로4길 21
none
이번에 찾을 키워드 : 3099 / 3192 행 갈치옥제주특별자치도 제주시 조천읍 조함해안로 530
https://pcmap.place.naver.com/restaurant/1876373893/review/visitor#
이번에 찾을 키워드 : 3100 / 3192 행 청년반점제주특별자치도 제주시 동문로4길 9
https://pcmap.place.naver.com/restaurant/1001501444/review/visitor#
이번에 찾을 키워드 : 3101 / 3192 행 떡보킹외도점제주특별자치도 제주시 우정로11길 18
https://pcmap.place.naver.com/restaurant/1599445047/review/visitor#
이번에 찾을 키워드 : 3102 / 3192 행 큰아들삼화점제주특별자치도 제주시 선사로8길 10-2
https://pcmap.place.naver.com/restaurant/1938659370/review/visitor#
이번에 찾을 키워드 : 3103 / 3192 행 청년피자제주외도점제주특별자치도 제주시 우정로 50
https://pcmap.place.naver.com/restaurant/1341857486/review/visitor#
이번에 찾을 키워드 : 3104 / 3192 행 명도암정담김밥제주특별자치도 제주시 명림로 236
https://pcmap.place.naver.com/restaurant/1343014715/review/visitor#
이번에 찾을 키워드 : 3105 / 3192 행 진돼지이도점제주특별자치도 제주시 정든로3길 38
https://pcmap.place.naver.c

https://pcmap.place.naver.com/restaurant/1161419654/review/visitor#
이번에 찾을 키워드 : 3175 / 3192 행 농부파스타집제주특별자치도 제주시 구좌읍 해맞이해안로 518
https://pcmap.place.naver.com/restaurant/1630075478/review/visitor#
이번에 찾을 키워드 : 3176 / 3192 행 루아르제주특별자치도 제주시 산지로 43
https://pcmap.place.naver.com/restaurant/1644897993/review/visitor#
이번에 찾을 키워드 : 3177 / 3192 행 작업실사월제주특별자치도 제주시 중앙로 57
none
이번에 찾을 키워드 : 3178 / 3192 행 제이앤제이홀덤펍제주특별자치도 제주시 서광로32길 39
https://pcmap.place.naver.com/restaurant/1227032733/review/visitor#
이번에 찾을 키워드 : 3179 / 3192 행 탕수육제주특별자치도 제주시 노형9길 8
https://pcmap.place.naver.com/restaurant/1793239925/review/visitor#
이번에 찾을 키워드 : 3180 / 3192 행 샘물카페제주특별자치도 제주시 천수로6길 3
https://pcmap.place.naver.com/restaurant/1052014961/review/visitor#
이번에 찾을 키워드 : 3181 / 3192 행 트립레시피제주특별자치도 제주시 애월읍 수산북길 9
https://pcmap.place.naver.com/restaurant/1360312564/review/visitor#
이번에 찾을 키워드 : 3182 / 3192 행 203안전한먹거리연구소제주특별자치도 제주시 선덕로 4
https://pcmap.place.naver.com/restaurant/1129882445/review/visitor#
이번에 찾을 키워드 : 3183 / 31

In [15]:
# for i, keyword in enumerate(df['사업장명'].tolist()):
#     print("이번에 찾을 키워드 :", i, f"/ {df.shape[0]} 행",keyword)
    
#     try:
#         naver_map_search_url = f"https://map.naver.com/v5/search/제주&{keyword}/place" # 검색 url 만들기
#         driver.get(naver_map_search_url) # 검색 url 접속, 즉 검색하기
        
#         time.sleep(2) # 중요함
#         cu = driver.current_url # 검색이 성공된 플레이스에 대한 개별 페이지
        
#         res_code = re.findall(r"place/(\d+)", cu)
#         final_url = 'https://pcmap.place.naver.com/restaurant/'+res_code[0]+'/review/visitor#'
        
#         print(final_url)
#         df['naver_map_url'][i]=final_url
        
#     except IndexError:
#         df['naver_map_url'][i]= ''
#         print('none')

In [17]:
# df
df['naver_map_url'].replace('', np.nan, inplace=True)

C:\Users\ui\anaconda3\lib\site-packages\pandas\core\series.py:4509: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().replace(


In [18]:
df = df.dropna(subset=['naver_map_url'])

In [19]:
df.to_csv('url_completed.csv', encoding = 'utf-8-sig')

In [20]:
df 

,사업장명,사업장명add,업종구분대분류,업종구분소분류,주소,naver_map_url
1,이스트힐스카페테리아,이스트힐스카페테리아제주특별자치도 제주시 조천읍 종인내길 323,일반음식점,한식,제주특별자치도 제주시 조천읍 종인내길 323,https://pcmap.place.naver.com/restaurant/13091...
3,큰맘할매순대국밥,큰맘할매순대국밥제주특별자치도 제주시 구남동5길 42 (아라이동),일반음식점,한식,제주특별자치도 제주시 구남동5길 42 (아라이동),https://pcmap.place.naver.com/restaurant/19874...
6,제주통큰장어,제주통큰장어제주특별자치도 제주시 청귤로3길 50 (이도이동),일반음식점,한식,제주특별자치도 제주시 청귤로3길 50 (이도이동),https://pcmap.place.naver.com/restaurant/32782...
8,휴식공간,휴식공간제주특별자치도 제주시 한경면 낙수로 85,일반음식점,한식,제주특별자치도 제주시 한경면 낙수로 85,https://pcmap.place.naver.com/restaurant/15657...
9,숙이네국수,숙이네국수제주특별자치도 제주시 고마로 29,일반음식점,한식,제주특별자치도 제주시 고마로 29,https://pcmap.place.naver.com/restaurant/36835...
...,...,...,...,...,...,...
3185,태봉이네,태봉이네제주특별자치도 제주시 원노형남1길 43,일반음식점,한식,제주특별자치도 제주시 원노형남1길 43,https://pcmap.place.naver.com/restaurant/38711...
3187,돈복편백,돈복편백제주특별자치도 제주시 애월읍 애월해안로 29,일반음식점,한식,제주특별자치도 제주시 애월읍 애월해안로 29,https://pcmap.place.naver.com/restaurant/17491...
3188,유로코피자제주2호점,유로코피자제주2호점제주특별자치도 제주시 애월읍 애월해안로 960,일반음식점,기타,제주특별자치도 제주시 애월읍 애월해안로 960,https://pcmap.place.naver.com/restaurant/16157...
3190,카페뷰,카페뷰제주특별자치도 제주시 우도면 우도해안길 684,일반음식점,기타,제주특별자치도 제주시 우도면 우도해안길 684,https://pcmap.place.naver.com/restaurant/16860...


In [21]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
import time
import re

from bs4 import BeautifulSoup
from tqdm import tqdm

# 웹드라이버 접속
driver = webdriver.Chrome(executable_path='./chromedriver.exe')

df = pd.read_csv('url_completed.csv') #null값 처리 완료 데이터

# 수집할 정보들
rating_list = [] # 평점
user_review_id = {} # 유저의 id - 추천시스템에서는 필수
review_json = {} # 리뷰
image_json = {} # 이미지

In [ ]:
# for i in range(len(df)):
    
#     print('======================================================')
#     print(str(i)+'번째 식당')
    
#     # 식당 리뷰 개별 url 접속
#     driver.get(df['naver_map_url'][i])
#     thisurl = df['naver_map_url'][i]
#     time.sleep(2)
#     # 더보기 버튼 다 누르기
#     # 더보기 버튼 누르기까지 10개
#     # 더보기 버튼 누르면 10개 추가됨
#     while True:
#         try:
#             time.sleep(2)
#             driver.find_element_by_tag_name('body').send_keys(Keys.END)
#             time.sleep(5)
#             driver.find_element_by_css_selector('#app-root > div > div.place_detail_wrapper > div:nth-child(5) > div:nth-child(4) > div:nth-child(4) > div._2kAri > a').click()
#             time.sleep(5)  -->이줄변경해야됨
#                    
#             driver.find_element_by_tag_name('body').send_keys(Keys.END)
#             time.sleep(2)
            
#         except NoSuchElementException:
#             print('-더보기 버튼 모두 클릭 완료-')
#             break

    
#     # 파싱
#     html = driver.page_source
#     soup = BeautifulSoup(html, 'lxml')
#     time.sleep(1)
    
#     # 식당 구분
#     restaurant_name = df['사업장명'][i]
#     print('식당 이름 : '+restaurant_name)
    
#     user_review_id[restaurant_name] = {}
#     review_json[restaurant_name] = {}
#     image_json[restaurant_name] = {}
    
#     try:
#         restaurant_classificaton = soup.find_all('span',attrs = {'class':'_3ocDE'})[0].text
#     except:
#         restaurant_classificaton = 'none'
#     print('식당 구분 : '+restaurant_classificaton)
#     print('----------------------------------------------')
    
#     try:
#         one_review = soup.find_all('div', attrs = {'class':'_1Z_GL'})
#         review_num = len(one_review) # 특정 식당의 리뷰 총 개수
#         print('리뷰 총 개수 : '+str(review_num))
    
#         # 리뷰 개수
#         for i in range(len(one_review)):
        
#             # user url
#             user_url = one_review[i].find('div', attrs = {'class':'_23Rml'}).find('a').get('href')
#             print('user_url = '+user_url)
            
#             # user url로부터 user code 뽑아내기
#             user_code = re.findall(r"my/(\w+)", user_url)[0]
#             print('user_code = '+user_code)
            
#             # review 1개에 대한 id 만들기 
#             res_code = re.findall(r"restaurant/(\d+)", thisurl)[0]
#             review_id = str(res_code)+"_"+user_code
#             print('review_id = '+review_id)
            
#             # rating, 별점
#             rating = one_review[i].find('span', attrs = {'class':'_2tObC'}).text
#             print('rating = '+rating)
            
#             # 주의!!! 사진 리뷰 유무에 따라 날짜 파싱코드 다름
#             # ('span', attrs = {'class':'_3WqoL'})
#             # 사진 없는 경우 : 총 6개 중 4번째
#             # 사진 있는 경우 : 총 5개 중 3번째
            
#             # date
#             # 사진 리뷰 없음
#             if len(one_review[i].find_all('span', attrs = {'class':'_3WqoL'})) == 5:
#                 date = one_review[i].find_all('span', attrs = {'class':'_3WqoL'})[2].text
#             elif len(one_review[i].find_all('span', attrs = {'class':'_3WqoL'})) == 6:
#                 date = one_review[i].find_all('span', attrs = {'class':'_3WqoL'})[3].text
#             else:
#                 date = ""
#             print('date = '+date)
            
#             # review 내용
#             try : 
#                 review_content = one_review[i].find('span', attrs = {'class':'WoYOw'}).text
#             except: # 리뷰가 없다면
#                 review_content = ""
#             print('리뷰 내용 : '+review_content)
            
            
#             # image 내용
#             sliced_soup = one_review[i].find('div', attrs = {'class':'_1aFEL _2GO1Q'})

#             if (sliced_soup != None):
#                 sliced_soup = sliced_soup.find('div',attrs={'class':'dRZ2X'})
                
#                 try:
#                     img_url = 'https://search.pstatic.net/common/?autoRotate=true&quality=95&type=l&size=800x800&src='+re.findall(r'src=(.*jpeg)', str(sliced_soup))[0]
#                 except :
#                     if (len(re.findall(r'src=(.*jpg)', str(sliced_soup)))!= 0):
#                         img_url = 'https://search.pstatic.net/common/?autoRotate=true&quality=95&type=l&size=800x800&src='+re.findall(r'src=(.*jpg)', str(sliced_soup))[0]
#                     elif (len(re.findall(r'src=(.*png)', str(sliced_soup)))!= 0):
#                         img_url = 'https://search.pstatic.net/common/?autoRotate=true&quality=95&type=l&size=800x800&src='+re.findall(r'src=(.*png)', str(sliced_soup))[0]
#                     else :
#                         img_url = ""
#             else:
#                 img_url = ""
                
#             print('이미지 url : '+img_url)
#             print('----------------------------------------------')
#             print('\n')

            
#             #리뷰정보
#             # user_review_id
#             user_review_id[restaurant_name][user_code] = review_id
            
#             # review_json
#             review_json[restaurant_name][review_id] = review_content
            
#             # image_json
#             image_json[restaurant_name][review_id] = img_url
            
#             # rating_df_list
#             naver_review = user_code, restaurant_name, rating, date
#             rating_list.append(naver_review)
            
            
    
    
#     except NoSuchElementException:
#             none_review = "네이버 리뷰 없음"
#             print(none_review)
#             review_num = 0
#             #리뷰정보 = restaurant_name, restaurant_classification, review_num, none_review
            
#             # rating_df_list
#             naver_review = user_code, restaurant_name, none_review, none_reivew
#             rating_list.append(naver_review)
            
            
#     print('\n')

In [ ]:
for i in range(839,1700): # 1회차  839~2093 준영


    print('======================================================')
    print(str(i)+'번째 식당')
    
    # 식당 리뷰 개별 url 접속
    driver.get(df['naver_map_url'][i])
    thisurl = df['naver_map_url'][i]
    time.sleep(2)
    # 더보기 버튼 다 누르기
    # 더보기 버튼 누르기까지 10개
    # 더보기 버튼 누르면 10개 추가됨
    while True:
        try:
            time.sleep(1) # 1초로 바꾸고
            driver.find_element_by_tag_name('body').send_keys(Keys.END)
            time.sleep(3.5) # 4초로 바꾸고
            driver.find_element_by_xpath('//*[@id="app-root"]/div/div/div[2]/div[5]/div[4]/div[4]/div[2]/a').click()
            #driver.find_element_by_xpath('//*[@id="app-root"]/div/div/div[2]/div[5]/div[4]/div[5]/div[2]/a').click()
            # driver.find_element_by_xpath('//*[@id="app-root"]/div/div/div[2]/div[5]/div[4]/div[4]/div[2]/a').click()리뷰안되면이걸로
            time.sleep(3.5) # 4초로 바꾸고
            driver.find_element_by_tag_name('body').send_keys(Keys.END)
            time.sleep(1)  # 1초로 바꾸고
            
        except NoSuchElementException:
            print('-더보기 버튼 모두 클릭 완료-')
            break

    
    # 파싱
    html = driver.page_source
    soup = BeautifulSoup(html, 'lxml')
    time.sleep(1)
    
    # 식당 구분
    restaurant_name = df['사업장명'][i]
    print('식당 이름 : '+restaurant_name)
    
    user_review_id[restaurant_name] = {}
    review_json[restaurant_name] = {}
    image_json[restaurant_name] = {}
    
    try:
        restaurant_classificaton = soup.find_all('span',attrs = {'class':'_3ocDE'})[0].text
    except:
        restaurant_classificaton = 'none'
    print('식당 구분 : '+restaurant_classificaton)
    print('----------------------------------------------')
    
    try:
        one_review = soup.find_all('div', attrs = {'class':'_1Z_GL'})
        review_num = len(one_review) # 특정 식당의 리뷰 총 개수
        print('리뷰 총 개수 : '+str(review_num))
    
        # 리뷰 개수
        for i in range(len(one_review)):
        
            # user url
            user_url = one_review[i].find('div', attrs = {'class':'_23Rml'}).find('a').get('href')
            print('user_url = '+user_url)
            
            # user url로부터 user code 뽑아내기
            user_code = re.findall(r"my/(\w+)", user_url)[0]
            print('user_code = '+user_code)
            
            # review 1개에 대한 id 만들기 
            res_code = re.findall(r"restaurant/(\d+)", thisurl)[0]
            review_id = str(res_code)+"_"+user_code
            print('review_id = '+review_id)
            
            # rating, 별점
            rating = one_review[i].find('span', attrs = {'class':'_2tObC'}).text
            print('rating = '+rating)
            
            # 주의!!! 사진 리뷰 유무에 따라 날짜 파싱코드 다름
            # ('span', attrs = {'class':'_3WqoL'})
            # 사진 없는 경우 : 총 6개 중 4번째
            # 사진 있는 경우 : 총 5개 중 3번째
            
            # date
            # 사진 리뷰 없음
            if len(one_review[i].find_all('span', attrs = {'class':'_3WqoL'})) == 5:
                date = one_review[i].find_all('span', attrs = {'class':'_3WqoL'})[2].text
            elif len(one_review[i].find_all('span', attrs = {'class':'_3WqoL'})) == 6:
                date = one_review[i].find_all('span', attrs = {'class':'_3WqoL'})[3].text
            else:
                date = ""
            print('date = '+date)
            
            # review 내용
            try : 
                review_content = one_review[i].find('span', attrs = {'class':'WoYOw'}).text
            except: # 리뷰가 없다면
                review_content = ""
            print('리뷰 내용 : '+review_content)
            
            
            # image 내용
            sliced_soup = one_review[i].find('div', attrs = {'class':'_1aFEL _2GO1Q'})

            if (sliced_soup != None):
                sliced_soup = sliced_soup.find('div',attrs={'class':'dRZ2X'})
                
                try:
                    img_url = 'https://search.pstatic.net/common/?autoRotate=true&quality=95&type=l&size=800x800&src='+re.findall(r'src=(.*jpeg)', str(sliced_soup))[0]
                except :
                    if (len(re.findall(r'src=(.*jpg)', str(sliced_soup)))!= 0):
                        img_url = 'https://search.pstatic.net/common/?autoRotate=true&quality=95&type=l&size=800x800&src='+re.findall(r'src=(.*jpg)', str(sliced_soup))[0]
                    elif (len(re.findall(r'src=(.*png)', str(sliced_soup)))!= 0):
                        img_url = 'https://search.pstatic.net/common/?autoRotate=true&quality=95&type=l&size=800x800&src='+re.findall(r'src=(.*png)', str(sliced_soup))[0]
                    else :
                        img_url = ""
            else:
                img_url = ""
                
            print('이미지 url : '+img_url)
            print('----------------------------------------------')
            print('\n')

            
            #리뷰정보
            # user_review_id
            user_review_id[restaurant_name][user_code] = review_id
            
            # review_json
            review_json[restaurant_name][review_id] = review_content
            
            # image_json
            image_json[restaurant_name][review_id] = img_url
            
            # rating_df_list
            naver_review = user_code, restaurant_name, rating, date
            rating_list.append(naver_review)
            
            
    
    
    except NoSuchElementException:
            none_review = "네이버 리뷰 없음"
            print(none_review)
            review_num = 0
            #리뷰정보 = restaurant_name, restaurant_classification, review_num, none_review
            
            # rating_df_list
            naver_review = user_code, restaurant_name, none_review, none_reivew
            rating_list.append(naver_review)
            
            
    print('\n')

839번째 식당
-더보기 버튼 모두 클릭 완료-
식당 이름 : 진영루
식당 구분 : 중식당
----------------------------------------------
리뷰 총 개수 : 0


840번째 식당
-더보기 버튼 모두 클릭 완료-
식당 이름 : 플로라
식당 구분 : 카페
----------------------------------------------
리뷰 총 개수 : 3
user_url = https://m.place.naver.com/my/5c191b412fb7256c53565dbf/review
user_code = 5c191b412fb7256c53565dbf
review_id = 13082080_5c191b412fb7256c53565dbf
rating = 4.5
date = 2021.01.05
리뷰 내용 : 굿굿
이미지 url : 
----------------------------------------------


user_url = https://m.place.naver.com/my/5c58619ec9bf0a199faf6cb2/review
user_code = 5c58619ec9bf0a199faf6cb2
review_id = 13082080_5c58619ec9bf0a199faf6cb2
rating = 3.5
date = 2020.11.07
리뷰 내용 : 룸이라 편하네요... 그리구 레몬에이드는 물에 비타민씨 타먹는 맛이에요....
이미지 url : 
----------------------------------------------


user_url = https://m.place.naver.com/my/5c06c5b850fa98499fbdebbb/review
user_code = 5c06c5b850fa98499fbdebbb
review_id = 13082080_5c06c5b850fa98499fbdebbb
rating = 1.5
date = 2020.09.14
리뷰 내용 : 아쉬워요
이미지 url : https://search.

rating = 4.5
date = 2021.01.01
리뷰 내용 : 좋습니다 맛 비주얼 분위기  새해 첫날부터 기분좋은 장소추천!!!
이미지 url : https://search.pstatic.net/common/?autoRotate=true&quality=95&type=l&size=800x800&src=https%3A%2F%2Fmyplace-phinf.pstatic.net%2F20210101_31%2F1609499741398akJ3O_JPEG%2Fupload_c8f97bbb4a979d8a92f71c1cfe2bab3e.jpg
----------------------------------------------


user_url = https://m.place.naver.com/my/5b7b775d47717e0a84ab7783/review
user_code = 5b7b775d47717e0a84ab7783
review_id = 37293973_5b7b775d47717e0a84ab7783
rating = 0.5
date = 2021.01.01
리뷰 내용 : 여기는..... 양에만 신경쓰셨음.... 양에 신경쓰기보단 맛에 신경좀 쓰셨으면 좋을듯요. 덮밥은 고추장 한숟가락에 설탕 뿌린맛.... 짬뽕은 짬뽕면으로 끓인 다소 싱거운맛.... 제 입맛이 예민한편은 아니지만 별 반개도 아까울정도예요. 1/3도 못먹고 나왔음
이미지 url : 
----------------------------------------------


user_url = https://m.place.naver.com/my/5c0ce1163b6c8f225dddace7/review
user_code = 5c0ce1163b6c8f225dddace7
review_id = 37293973_5c0ce1163b6c8f225dddace7
rating = 5
date = 2020.12.15
리뷰 내용 : 짬뽕매워요 근데맛있음요
이미지 url : 
-------------------------------------

rating = 5
date = 2020.07.30
리뷰 내용 : 인생맛집이에요 ㅋㅋㅋ
해장으로 최고예요 양도 푸짐하고 재료 하나하나가 신선해요
이미지 url : 
----------------------------------------------


user_url = https://m.place.naver.com/my/5efde1ec67c6d4320f56cbb0/review
user_code = 5efde1ec67c6d4320f56cbb0
review_id = 37293973_5efde1ec67c6d4320f56cbb0
rating = 5
date = 2020.07.30
리뷰 내용 : 원래 짬뽕은 면이 싱거워서 별로 안좋아하는데 여기 짬뽕 너무 맛있게 먹고왔어요 국물은 그냥 입댈것도 없이 맛있고  면도 탱글탱글 
간이 딱 맞고 최고예요
이미지 url : 
----------------------------------------------


user_url = https://m.place.naver.com/my/5d41cc5f0be8e710b9212f54/review
user_code = 5d41cc5f0be8e710b9212f54
review_id = 37293973_5d41cc5f0be8e710b9212f54
rating = 5
date = 2020.07.29
리뷰 내용 : 제주도 여행와서 대만족스러운 식사였어요 
문어 진짜 쫄깃하고 존맛탱 ..
서울가면 또 생각날 맛이에요ㅠㅠㅠo̴̶̷᷄﹏o̴̶̷̥᷅
이미지 url : 
----------------------------------------------


user_url = https://m.place.naver.com/my/5ec5df8a8f87a842bcfbeaea/review
user_code = 5ec5df8a8f87a842bcfbeaea
review_id = 37293973_5ec5df8a8f87a842bcfbeaea
rating = 5
date = 2020.07.29
리뷰 내용 : 반찬도 깔

-더보기 버튼 모두 클릭 완료-
식당 이름 : 낭랑
식당 구분 : 해물,생선요리
----------------------------------------------
리뷰 총 개수 : 1
user_url = https://m.place.naver.com/my/5eff2a977a3924f9464c79f6/review
user_code = 5eff2a977a3924f9464c79f6
review_id = 32162592_5eff2a977a3924f9464c79f6
rating = 5
date = 2020.12.18
리뷰 내용 : #제주현지인맛집#아나고맛집#제주아나고구이#제주맛집#제주공항맛집#믿고가는맛집
이미지 url : https://search.pstatic.net/common/?autoRotate=true&quality=95&type=l&size=800x800&src=https%3A%2F%2Fmyplace-phinf.pstatic.net%2F20201219_226%2F1608355927781E5WUW_PNG%2Fupload_2a836c158202b73d570edab3b9201e3f.png
----------------------------------------------




845번째 식당
-더보기 버튼 모두 클릭 완료-
식당 이름 : 봉이네
식당 구분 : 돼지고기구이
----------------------------------------------
리뷰 총 개수 : 21
user_url = https://m.place.naver.com/my/5b9b8764bbbc02d6b331f40e/review
user_code = 5b9b8764bbbc02d6b331f40e
review_id = 16887211_5b9b8764bbbc02d6b331f40e
rating = 5
date = 2021.08.27
리뷰 내용 : 고기 굽느라 집에 냄새 안풍기고 너무 좋네요.
이미지 url : 
----------------------------------------------

-더보기 버튼 모두 클릭 완료-
식당 이름 : 은성
식당 구분 : 생선회
----------------------------------------------
리뷰 총 개수 : 5
user_url = https://m.place.naver.com/my/5bb40fa346a2164913804541/review
user_code = 5bb40fa346a2164913804541
review_id = 13110864_5bb40fa346a2164913804541
rating = 5
date = 2021.10.05
리뷰 내용 : 굿
이미지 url : 
----------------------------------------------


user_url = https://m.place.naver.com/my/5f44a2ef7ecfbfb7b011ad8b/review
user_code = 5f44a2ef7ecfbfb7b011ad8b
review_id = 13110864_5f44a2ef7ecfbfb7b011ad8b
rating = 4.5
date = 2021.08.27
리뷰 내용 : 
이미지 url : https://search.pstatic.net/common/?autoRotate=true&quality=95&type=l&size=800x800&src=https%3A%2F%2Fmyplace-phinf.pstatic.net%2F20210828_103%2F1630078203900G9eAm_JPEG%2Fupload_9e7043346f5fdafe205c2376c19ea5c8.jpg
----------------------------------------------


user_url = https://m.place.naver.com/my/5b9c6ee54754394dc82ba1ad/review
user_code = 5b9c6ee54754394dc82ba1ad
review_id = 13110864_5b9c6ee54754394dc82ba1ad
rating = 5
date = 2021.0

...

-더보기 버튼 모두 클릭 완료-
식당 이름 : 빅토리아
식당 구분 : 카페,디저트
----------------------------------------------
리뷰 총 개수 : 25
user_url = https://m.place.naver.com/my/60efed753449bd59e074f583/review
user_code = 60efed753449bd59e074f583
review_id = 1848762781_60efed753449bd59e074f583
rating = 5
date = 2021.10.18
리뷰 내용 : 굳
이미지 url : 
----------------------------------------------


user_url = https://m.place.naver.com/my/5f94162a150ad938202692de/review
user_code = 5f94162a150ad938202692de
review_id = 1848762781_5f94162a150ad938202692de
rating = 5
date = 2021.10.18
리뷰 내용 : 쉬기 정말 좋네요.
이미지 url : 
----------------------------------------------


user_url = https://m.place.naver.com/my/5dd8b7869ec8258e4adcc7c7/review
user_code = 5dd8b7869ec8258e4adcc7c7
review_id = 1848762781_5dd8b7869ec8258e4adcc7c7
rating = 5
date = 2021.08.29
리뷰 내용 : 무인인데좋아요
이미지 url : 
----------------------------------------------


user_url = https://m.place.naver.com/my/5c1c1849d77f6e947822e5a2/review
user_code = 5c1c1849d77f6e947822e5a2
re

-더보기 버튼 모두 클릭 완료-
식당 이름 : 아싸살아있네
식당 구분 : 생선회
----------------------------------------------
리뷰 총 개수 : 26
user_url = https://m.place.naver.com/my/5e40a95c8f87a842bc143f96/review
user_code = 5e40a95c8f87a842bc143f96
review_id = 1405097012_5e40a95c8f87a842bc143f96
rating = 4
date = 2021.09.09
리뷰 내용 : 주변 찬들이 많고 맛있어서 
2박 3일 동안 2번 먹었어요!
이미지 url : 
----------------------------------------------


user_url = https://m.place.naver.com/my/5e091fe58f87a842bc3806c4/review
user_code = 5e091fe58f87a842bc3806c4
review_id = 1405097012_5e091fe58f87a842bc3806c4
rating = 4
date = 2021.08.05
리뷰 내용 : 사람많아요~한치 싱싱해요
이미지 url : https://search.pstatic.net/common/?autoRotate=true&quality=95&type=l&size=800x800&src=https%3A%2F%2Fmyplace-phinf.pstatic.net%2F20210821_48%2F1629506675132NoNAv_JPEG%2Fupload_242dc49d977f6988385e39af27b401b6.jpg
----------------------------------------------


user_url = https://m.place.naver.com/my/5db9c7748f87a842bc3f66d9/review
user_code = 5db9c7748f87a842bc3f66d9
review_id = 1405097

-더보기 버튼 모두 클릭 완료-
식당 이름 : 섬집
식당 구분 : 생선회
----------------------------------------------
리뷰 총 개수 : 3
user_url = https://m.place.naver.com/my/5dd0a34d8f87a842bc0755a1/review
user_code = 5dd0a34d8f87a842bc0755a1
review_id = 1423132555_5dd0a34d8f87a842bc0755a1
rating = 5
date = 2021.06.04
리뷰 내용 : 좋아요
이미지 url : 
----------------------------------------------


user_url = https://m.place.naver.com/my/5ba5b7892a73bbb8f7aad9f7/review
user_code = 5ba5b7892a73bbb8f7aad9f7
review_id = 1423132555_5ba5b7892a73bbb8f7aad9f7
rating = 5
date = 2021.03.24
리뷰 내용 : 너무 맛있어요 ㅠㅠ같이 먹는 갓김치도 짱이에요 ㅠㅠ 술이 술술술 ㅠㅠ  너무 맛있고 사장님도 너무 친절해요 ㅋ
이미지 url : https://search.pstatic.net/common/?autoRotate=true&quality=95&type=l&size=800x800&src=https%3A%2F%2Fmyplace-phinf.pstatic.net%2F20210324_171%2F1616588816824j9zHV_JPEG%2Fupload_15780a001e13422fc9316b1f72e07572.jpeg
----------------------------------------------


user_url = https://m.place.naver.com/my/5f0ea0379ec8258e4a2c7588/review
user_code = 5f0ea0379ec8258e4a2c7588
revi

-더보기 버튼 모두 클릭 완료-
식당 이름 : 신해바라기제2호점
식당 구분 : 한식
----------------------------------------------
리뷰 총 개수 : 17
user_url = https://m.place.naver.com/my/5da43f4e8f87a842bcb27573/review
user_code = 5da43f4e8f87a842bcb27573
review_id = 31225280_5da43f4e8f87a842bcb27573
rating = 5
date = 2021.09.03
리뷰 내용 : 좋아요!
이미지 url : 
----------------------------------------------


user_url = https://m.place.naver.com/my/5f043949e9d94998c6cc71bf/review
user_code = 5f043949e9d94998c6cc71bf
review_id = 31225280_5f043949e9d94998c6cc71bf
rating = 5
date = 2021.06.22
리뷰 내용 : 굿
이미지 url : 
----------------------------------------------


user_url = https://m.place.naver.com/my/5cceaa012d43c5afab08673f/review
user_code = 5cceaa012d43c5afab08673f
review_id = 31225280_5cceaa012d43c5afab08673f
rating = 4
date = 2021.06.06
리뷰 내용 : 후기보고 기대안햇는데 기대이싱
이미지 url : 
----------------------------------------------


user_url = https://m.place.naver.com/my/5ce4e544c3e0e5dabbba56d8/review
user_code = 5ce4e544c3e0e5dabbba56d8
revi

-더보기 버튼 모두 클릭 완료-
식당 이름 : 나이스
식당 구분 : 한식
----------------------------------------------
리뷰 총 개수 : 0


918번째 식당
-더보기 버튼 모두 클릭 완료-
식당 이름 : 자순
식당 구분 : 한식
----------------------------------------------
리뷰 총 개수 : 9
user_url = https://m.place.naver.com/my/5deef5038f87a842bced4f0c/review
user_code = 5deef5038f87a842bced4f0c
review_id = 16814772_5deef5038f87a842bced4f0c
rating = 5
date = 2021.04.26
리뷰 내용 : 방문
이미지 url : 
----------------------------------------------


user_url = https://m.place.naver.com/my/5c9c11438f865fe934386d66/review
user_code = 5c9c11438f865fe934386d66
review_id = 16814772_5c9c11438f865fe934386d66
rating = 3.5
date = 2021.04.26
리뷰 내용 : 방문인증합니다
이미지 url : 
----------------------------------------------


user_url = https://m.place.naver.com/my/5dc4dd0e8f87a842bce0fc55/review
user_code = 5dc4dd0e8f87a842bce0fc55
review_id = 16814772_5dc4dd0e8f87a842bce0fc55
rating = 4.5
date = 2021.03.06
리뷰 내용 : 찌개류와 비빔밥 모두 맛이있고 편하게 식사하기 좋아서 한번씩 찾는 곳입니다.
아이랑 가면 김도 내어주시고 계란말이나 후라이 메뉴가 있어서 좋아

In [49]:
rating_df = pd.DataFrame(rating_list)
rating_df.columns = ['UserID','ItemID','Rating','Timestamp']
rating_df.to_csv('rating9.csv', encoding='utf-8-sig')

import json

file_path = "./user_review_id.json"
with open(file_path,'w', encoding="UTF-8") as outfile:
    json.dump(user_review_id, outfile, ensure_ascii=False)
    
file_path = "./review.json"
with open(file_path,'w', encoding="UTF-8") as outfile:
    json.dump(review_json, outfile, ensure_ascii=False)

file_path = "./image.json"
with open(file_path,'w', encoding="UTF-8") as outfile:
    json.dump(image_json, outfile, ensure_ascii=False)

## 최종 크롤링된 데이터프레임에
## 공통 값을 기준으로(열 기준) concat하여 주소 붙이기 !!!

In [ ]:
# 수정사항
# 1. 1번파일에 상호명+도로명전체주소 처리한 새로운 열만들어서-해결함
# 2. 이 새로운 열로 검색 url키워드 만들기-해결함
# 3. 만들어진 키워드로 검색-해결함